In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!rm -rf input_docs
!cp /content/drive/MyDrive/input_docs_sample.zip . # Sample data 
#!cp /content/drive/MyDrive/input_docs.zip . (Real data)
!unzip input_docs_sample.zip > /dev/null
#!unzip input_docs.zip > /dev/null
!ls input_docs/ | wc -l

5


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [ ]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os

#(did,text)
final_input = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

print(final_input.take(1))

[(4, '<H2>26-FEB-1987 15:07:13.72</H2>\r\n<H2>TALKING POINT/BANKAMERICA BAC EQUITY OFFER</H2>\r\nBankAmerica Corp is not under\npressure to act quickly on its proposed equity offering and\nwould do well to delay it because of the stock\'s recent poor\nperformance, banking analysts said.\n    Some analysts said they have recommended BankAmerica delay\nits up to one-billion-dlr equity offering, which has yet to be\napproved by the Securities and Exchange Commission.\n    BankAmerica stock fell this week, along with other banking\nissues, on the news that Brazil has suspended interest payments\non a large portion of its foreign debt.\n    The stock traded around 12, down 1/8, this afternoon,\nafter falling to 11-1/2 earlier this week on the news.\n    Banking analysts said that with the immediate threat of the\nFirst Interstate Bancorp I takeover bid gone, BankAmerica is\nunder no pressure to sell the securities into a market that\nwill be nervous on bank stocks in the near term.\n    Ban

In [ ]:
nltk.download('punkt')
from bs4 import BeautifulSoup
import collections
from collections import Counter
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stem = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def dw(docid, htmltext):

  lower_html = htmltext.lower() #convert into lowercase
  clean_text = BeautifulSoup(lower_html).get_text()
  
  clean_text = re.sub("\d+", " ", clean_text) #remove digits
  clean_text = re.sub('[^A-Za-z0-9]+', ' ', clean_text) #remove special characters
  clean_text = re.sub("\s+", " ", clean_text) #remove extra spaces
  
  clean_text = word_tokenize(clean_text)
  clean_text = [word for word in clean_text if word not in stopwords.words('english')]
  
  output = [];
  counter = Counter(clean_text)
  maxf = Counter(clean_text).most_common(1)[0][1]
  for ele in counter:
    tuple1 = (ele);
    tuple2 = (int(docid),counter[ele],counter[ele]/maxf)
    final_tuple = (tuple1, tuple2)
    output.append(final_tuple);
  return(output)

word_docid_freq_tf = final_input.flatMap(lambda x: dw(x[0],x[1]))
print(word_docid_freq_tf.take(2))

[('feb', (4, 1, 0.07142857142857142)), ('talking', (4, 1, 0.07142857142857142))]


In [ ]:
%xmode Verbose
#combining all the words and documents ids frequency
word_list_freq_tf = word_docid_freq_tf.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x+y)
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])
word_postinglist_freq_tf = sc.parallelize(word_list_freq_tf.collect())
#print(word_postinglist_freq_tf.count())
print(word_postinglist_freq_tf.map(lambda x : (x[0], list(x[1]))).take(1))

Exception reporting mode: Verbose
[('feb', [(4, 1, 0.07142857142857142), (5, 1, 0.16666666666666666), (1, 1, 0.07142857142857142), (2, 1, 0.2), (3, 1, 0.3333333333333333)])]


In [ ]:
# idf = 1/len(postinglist_tf)
%xmode Verbose
%time
import math
from decimal import Decimal
with_tfidf = []
for row in word_postinglist_freq_tf.collect():
  idf = 1/len(row[1]) #present in number of documents
  backto_tuple = []
  for eachrow in row[1]:
    list_row = list(eachrow) #convert to list
    tfidf_val = Decimal((list_row[2]*idf)) #tf*idf
    list_row[2] ="{:.18f}".format(float(tfidf_val))
    backto_tuple.append(tuple(list_row))

  eachtuple = (row[0], backto_tuple)
  with_tfidf.append(eachtuple);
# print(with_tfidf[0])
# creating a RDD
word_postinglist_freq_tfidf = sc.parallelize(with_tfidf)

print(word_postinglist_freq_tfidf.take(2))
#print(word_postinglist_freq_tfidf.count())

Exception reporting mode: Verbose
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs
[('feb', [(4, 1, '0.014285714285714285'), (5, 1, '0.033333333333333333'), (1, 1, '0.014285714285714285'), (2, 1, '0.040000000000000008'), (3, 1, '0.066666666666666666')]), ('talking', [(4, 1, '0.071428571428571425')])]



**Calculating Magnitude**




In [ ]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 
%xmode Verbose
import math
from decimal import Decimal

convert_to_list = word_postinglist_freq_tfidf.collect() #make a list
convert_to_list = list(map(list, convert_to_list)) 
new_list = []
for val in convert_to_list: 
  for row in val[1]:
    row = list(row)
    # print(type(row[2]))
    magnitude = Decimal((float(row[2])**2))
    magnitude ="{:.18f}".format(float(magnitude))
    row = (row[0],(row[1],magnitude))
  new_list.append(row)

# RDD of (did,(freq,tfidf)) tuples
# creating a RDD
did_freq_tfidfsq_rdd = sc.parallelize(new_list)

print(did_freq_tfidfsq_rdd.take(2))
new_list1 = did_freq_tfidfsq_rdd.reduceByKey(lambda x,y: (max(x[0],y[0]),(float(x[1])+float(y[1]))))

# Produce (did,(maxf,magnitudesq))
# creating a RDD
doc_maxf_mag = sc.parallelize(new_list1.collect())

#print(doc_maxf_mag.count())
print(doc_maxf_mag.take(2))

Exception reporting mode: Verbose
[(3, (1, '0.004444444444444444')), (4, (1, '0.005102040816326530'))]
[(4, (14, 2.7653061224489752)), (2, (5, 3.8700000000000014))]


In [ ]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [ ]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [ ]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/My\ Drive/inv_idx.txt
!wc -l /content/drive/My\ Drive/inv_idx.txt

total 24
-rw-r--r-- 1 root root  9670 Apr 12 19:39 part-00001
-rw-r--r-- 1 root root 10042 Apr 12 19:39 part-00000
-rw-r--r-- 1 root root     0 Apr 12 19:39 _SUCCESS
('inc', [(2, 1, '0.100000000000000006'), (3, 1, '0.166666666666666657')])
('plan', [(2, 1, '0.100000000000000006'), (3, 1, '0.166666666666666657')])
('money', [(2, 1, '0.200000000000000011')])
('activities', [(2, 1, '0.200000000000000011')])
('companies', [(2, 1, '0.200000000000000011')])
('subsidiary', [(2, 1, '0.200000000000000011')])
('plc', [(2, 1, '0.200000000000000011')])
('owns', [(2, 1, '0.200000000000000011')])
('pct', [(2, 1, '0.200000000000000011')])
('operated', [(2, 1, '0.200000000000000011')])
198 inv_idx/part-00000
198 inv_idx/part-00001
396 /content/drive/My Drive/inv_idx.txt


In [ ]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/My\ Drive/doc_mag.txt
!wc -l /content/drive/My\ Drive/doc_mag.txt

total 8
-rw-r--r-- 1 root root 59 Apr 12 19:39 part-00000
-rw-r--r-- 1 root root 84 Apr 12 19:39 part-00001
-rw-r--r-- 1 root root  0 Apr 12 19:39 _SUCCESS
(4, (14, 2.7653061224489752))
(2, (5, 3.8700000000000014))
2 doc_mag/part-00000
3 doc_mag/part-00001
5 /content/drive/My Drive/doc_mag.txt
